## Ablation experiments 1
This notebook contains some code that can be used for performing the ablation experiments showed on section 3.2.3

In [9]:
import sys
sys.path.append("./functions")

from utils import *
from model_bet_varying_layers import *
from layer import *
import pandas as pd

### Training and testing the model when varying  the  number of layers

In [ ]:
param = { 
    "graph_files": ["ER_15_graphs_1000_500_nodes",#["ER_15_graphs_10000_5000_nodes",
                    "SF_15_graphs_1000_500_nodes"],# "SF_15_graphs_10000_5000_nodes",
                    # "GRP_15_graphs_10000_5000_nodes"],
    "generation_seeds": [10],
    "num_train" : 5,
    "num_copies": [10],#[100],
    "adj_size" : 10000,
    "split_seeds": [10],
    'model_seed' : 15,
    "num_epochs": 15,    
    "num_test" : 10,
}


results = {'g': [], 'layers':[], 'epoch':[], 'kt':[], 'std':[]}

for graph_file in param["graph_files"]:
    for genseed in param["generation_seeds"]:
        for num_copies in param["num_copies"]:
            for splitseed in param["split_seeds"]:
                for num_layers in range(1,8):

                    data_path_test = f"{graph_file}_{genseed}_genseed_{param['num_test']}_test_{param['adj_size']}_size_{splitseed}_splitseed.pickle"
                    #Load test data
                    with open("./data_splits/test/"+data_path_test,"rb") as fopen:
                        list_graph_test,list_n_seq_test,list_num_node_test,bc_mat_test,deg_mat_test = pickle.load(fopen)        

                    data_path_train = f"{graph_file}_{genseed}_genseed_{param['num_train']}_train_{num_copies}_copies_{param['adj_size']}_size_{splitseed}_splitseed.pickle"

                    #Load training data
                    print(f"Loading data...")
                    with open("./data_splits/train/"+data_path_train,"rb") as fopen:
                        list_graph_train,list_n_seq_train,list_num_node_train,bc_mat_train,deg_mat_train = pickle.load(fopen)

                    model_size = bc_mat_train.shape[0]

                    #Get adjacency matrices from graphs
                    print(f"Graphs to adjacency conversion.")

                    list_adj_train,list_adj_t_train = graph_to_adj_bet(list_graph_train,list_n_seq_train,list_num_node_train,model_size)
                    list_adj_test,list_adj_t_test = graph_to_adj_bet(list_graph_test,list_n_seq_test,list_num_node_test,model_size)


                    #Model parameters
                    torch.manual_seed(param["model_seed"])

                    hidden = 20

                    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

                    print(f"GNN_Bet{num_layers}")
                    exec(f'model = GNN_Bet{num_layers}(ninput=model_size,nhid=hidden,dropout=0.6)')
                    model.to(device)

                    optimizer = torch.optim.Adam(model.parameters(),lr=0.0005)
                    num_epoch = param["num_epochs"]

                    print("Training")
                    print(f"Total Number of epoches: {num_epoch}")
                    for e in range(num_epoch):
                        print(f"Epoch number: {e+1}/{num_epoch}")
                        train(list_adj_train,list_adj_t_train,list_num_node_train,bc_mat_train,model=model,device=device,optimizer=optimizer,size=model_size)

                        #to check test loss while training
                        with torch.no_grad():
                            r = test(list_adj_test,list_adj_t_test,list_num_node_test,bc_mat_test,deg_mat_test,model=model,device=device,size=model_size)
                        
                            results['g'].append(data_path_train)
                            results['layers'].append(num_layers)
                            results['epoch'].append(e)
                            results['kt'].append(r["kt"])
                            results['std'].append(r["std"])

                        df = pd.DataFrame.from_dict(results)
                        df.to_csv("./outputs/output_ablationtest_varying_layers_given.csv")